In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from google.colab import files

In [3]:
#path where the csv is located
path = "/content/drive/MyDrive/"

In [4]:
# import all packagees
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.naive_bayes import MultinomialNB

import nltk # it is the one of most comman libraries for Natural Language Process
import re # Regular Expression library

In [5]:
# importing the csv file from google drive
with open(path + "gender-classifier-data.csv", encoding="latin1", errors='ignore') as f:
  twitter_data = pd.read_csv(f)

In [6]:
#Use only the gender and text column
data = pd.concat([twitter_data.gender, twitter_data.text], axis=1)

In [7]:
#Drop Brand
data = data[data["gender"] != "brand"]
data = data[data["gender"] != "unknown"]

In [8]:
import nltk # natural language took kit
nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [9]:
# genders have two options (male/female). so i changed male to 0 and female to 1.
data.gender = [1 if gender == "female" else 0 for gender in data.gender]

In [ ]:
descript_list = []
for descript in data.text:
    descript = re.sub("[^a-zA-Z]", " ", descript)
    descript= descript.lower()
    descript = nltk.word_tokenize(descript)
    lemma = nltk.WordNetLemmatizer()
    descript = [lemma.lemmatize(word) for word in descript]
    descript = " ".join(descript)
    descript_list.append(descript)

In [ ]:
#TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
max_features = 5000
ti = TfidfVectorizer(stop_words = "english", ngram_range=(1, 2))
sparce_matrix_ti = ti.fit_transform(descript_list).toarray()
print("top used {} words: {}".format(max_features, ti.get_feature_names()))

In [ ]:
# get column 0, which is the gender as an NDARRAY of shape ( n,) 
y = data.iloc[:, 0].values
x = sparce_matrix_ti

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif,f_classif
feature_selector = SelectKBest(score_func=chi2, k=1000).fit(sparce_matrix_ti, y)
# get best feature names
important_feature_names = feature_selector.get_feature_names_out(ti.get_feature_names_out())
name_and_scores = list(zip(important_feature_names, feature_selector.scores_))
name_and_scores.sort(key=lambda x: x[1], reverse=True)
for ns in name_and_scores:
  print(ns)

In [ ]:
# now, we will implement "bag of words" method

from sklearn.feature_extraction.text import CountVectorizer

max_features = 5000 # we will take top 5000 feature,means it will give 5000 most common words 

cv = CountVectorizer(max_features=max_features, stop_words = "english")
# in this method, we remove the stopwords (irrelevant words) in English language. (like "of", "and", "the" etc.)

sparce_matrix = cv.fit_transform(descript_list).toarray()

print("top used {} words: {}".format(max_features, cv.get_feature_names()))

In [ ]:
# get column 0, which is the gender as an NDARRAY of shape ( n,) 
y = data.iloc[:, 0].values
x = sparce_matrix_ti
print(sparce_matrix_ti)

NameError: ignored

In [ ]:
# cross validation, use 10% of the data to test;42 is a common choice to decide the seed of randomness
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(feature_selector.transform(sparce_matrix_ti), y, test_size = 0.1, random_state = 42)
print(y_test)

NameError: ignored

In [ ]:
#Import svm model
from sklearn import svm
from sklearn import metrics

#Classifier
svm_classifier = svm.SVC(kernel='linear') # Linear Kernel

#Train
svm_classifier.fit(x_train, y_train)

#Predict
y_pred = svm_classifier.predict(x_test)

# SVM accuacy
print("SVM Accuracy (text):",metrics.accuracy_score(y_test, y_pred))

In [ ]:
# use svm on description col
data_descript = pd.concat([twitter_data.gender, twitter_data.description], axis=1)


In [ ]:
#Drop Brand and unknown
data_descript = data_descript[data_descript["gender"] != "brand"]
data_descript = data[data["gender"] != "unknown"]

In [ ]:
# genders have two options (male/female). so i changed male to 0 and female to 1.
data_descript.gender = [1 if gender == "female" else 0 for gender in data.gender]

In [ ]:
import nltk
descript_list = []
for descript in data_descript.descript:
    descript = re.sub("[^a-zA-Z]", " ", descript)
    descript = descript.lower()
    descript = nltk.word_tokenize(descript)
    lemma = nltk.WordNetLemmatizer()
    descript = [lemma.lemmatize(word) for word in descript]
    descript = " ".join(descript)
    descript_list.append(descript)

In [ ]:
#TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
max_features = 5000
#vectorizer = TfidfVectorizer()
# ti = TfidfVectorizer(max_features = max_features, stop_words = "english")
ti = TfidfVectorizer(stop_words = "english", ngram_range=(1, 2))
sparce_matrix_ti = ti.fit_transform(description_list).toarray()
print("top used {} words: {}".format(max_features, ti.get_feature_names()))

In [ ]:
y = data.iloc[:, 0].values
x = sparce_matrix_ti

In [ ]:
# Feature extraction
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif,f_classif
# selected_features = SelectKBest(score_func=chi2, k=1000).fit_transform(sparce_matrix_ti, y)
feature_selector = SelectKBest(score_func=chi2, k=1000).fit(sparce_matrix_ti, y)
# get best feature names
important_feature_names = feature_selector.get_feature_names_out(ti.get_feature_names_out())
name_and_scores = list(zip(important_feature_names, feature_selector.scores_))
name_and_scores.sort(key=lambda x: x[1], reverse=True)
#for ns in name_and_scores:
#  print(ns)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

max_features = 5000 # we will take top 5000 feature,means it will give 5000 most common words 

cv = CountVectorizer(max_features=max_features, stop_words = "english")
# in this method, we remove the stopwords (irrelevant words) in English language. (like "of", "and", "the" etc.)

sparce_matrix = cv.fit_transform(description_list).toarray()

print("top used {} words: {}".format(max_features, cv.get_feature_names()))

In [ ]:
y = data.iloc[:, 0].values
x = sparce_matrix_ti
print(sparce_matrix_ti)

In [ ]:
# cross validation, use 10% of the data to test;42 is a common choice to decide the seed of randomness
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(feature_selector.transform(sparce_matrix_ti), y, test_size = 0.1, random_state = 42)
print(y_test)

In [ ]:
#Import svm model
from sklearn import svm
from sklearn import metrics

#Classifier
svm_classifier = svm.SVC(kernel='linear') # Linear Kernel

#Train
svm_classifier.fit(x_train, y_train)

#Predict
y_pred = svm_classifier.predict(x_test)

# SVM accuacy
print("SVM Accuracy (description):",metrics.accuracy_score(y_test, y_pred))